# OpenAI tools

With LCEL we can easily construc

In [1]:
# ! pip install -U openai duckduckgo-search

## Initialize tools

We will first create some tools we can use

In [2]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools.render import format_tool_to_openai_tool

In [3]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106")
tools = [DuckDuckGoSearchRun()]

## Using LCEL

We will first use LangChain Expression Language to create this agent

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [6]:
llm_with_tools = llm.bind(tools=[format_tool_to_openai_tool(t)for t in tools])

In [7]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [8]:
from langchain.agents import AgentExecutor

In [9]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [10]:
agent_executor.invoke(
    {
        "input": "What's the weather in LA, NYC, and SF today"
    }
)



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `weather in Los Angeles today`


60 °F like 60° Clear N 0 Today's temperature is forecast to be NEARLY THE SAME as yesterday. Radar Satellite WunderMap |Nexrad Today Wed 11/08 High 78 °F 0% Precip. / 0.00 in Sunny. High 78F.... Current Conditions Radar Forecasts Rivers and Lakes Climate and Past Weather Local Programs Fire Weather Show Caption Click a location below for detailed forecast. Last Map Update: Tue, Nov. 7, 2023 at 5:03:23 pm PST Watches, Warnings & Advisories Zoom Out Gale Warning Small Craft Advisory Wind Advisory Fire Weather Watch LOS ANGELES (KABC) -- Southern California will see mostly clear skies and mild temperatures on Monday. Los Angeles and Orange counties will see a few clouds in the morning, but they'll clear up... Storm No. 1: The first storm on Saturday will move quickly through Southern California, with most of the rain falling Saturday night. Light rain is possible late Friday for th

{'input': "What's the weather in LA, NYC, and SF today",
 'output': "Here's the weather for today in Los Angeles, New York City, and San Francisco:\n\n- Los Angeles: The temperature is around 60°F with clear skies and mild temperatures. There are no precipitation forecasts for today.\n- New York City: The temperature is around 68°F during the day and will drop to 48°F in the evening and night. There is no chance of rain forecasted for today.\n- San Francisco: The temperature is expected to reach a high of 67°F with sunny skies. Clouds are expected to increase over some areas around the bay during the day.\n\nIf you need more detailed information, feel free to ask!"}